# Generalised Archetype Analysis

This work is heavily adapted from the Archetype Analysis [repo](https://github.com/aleixalcacer/archetypes) by `aleixalcacer`. The goal of this Jupyter Notebook is to extend this work so that Archetype Analysis can perform on non-numeric data types, such as Boolean and ordinal data. The loss functions used to train the model are defined by Udell *et al.* in their work *Generalized Low-Rank Models*.

The archetype analysis problem for real-valued data is to minimise the residual sum of squares
$$L_{\text{real}} = \sum_{i=1}^n \left \lVert \mathbf{x}_i - \sum_{k=1}^p \alpha_{ik} \sum_{j=1}^n \beta_{kj} \mathbf{x}_j \right \rVert^2$$
for $\alpha$ and $\beta$.

For Boolean data, I suspect that the archetype analysis problem is to minimise the hinge loss
$$L_{\text{bool}} = \sum_{i=1}^n \left(1 - \mathbf{x}_i \sum_{k=1}^p \alpha_{ik} \sum_{j=1}^n \beta_{kj} \mathbf{x}_j \right)_+$$
akin to the Boolean PCA loss function defined in *Generalized Low-Rank Models*.

For ordinal data, I suspect that the archetype analysis problem is to minimise the modified hinge loss
$$L_{\text{ord}} = \sum_{i=1}^n \left(\sum_{a'=1}^{\mathbf{x}_i - 1} \left(1 - \sum_{k=1}^p \alpha_{ik} \sum_{j=1}^n \beta_{kj} \mathbf{x}_j + a' \right)_+ + \sum_{a' = \mathbf{x}_i + 1}^d \left(1 + \sum_{k=1}^p \alpha_{ik} \sum_{j=1}^n \beta_{kj} \mathbf{x}_j - a' \right)_+ \right)$$
corresponding to Equation (19) in *Generalized Low-Rank Models*.

In [2]:
# Libraries to import
from math import inf
from scipy.optimize import nnls
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted, check_random_state

import numpy as np
import pandas as pd

path_to_data = '~/honours/data/adult/adult.data' # change this to where your copy of the Adult dataset is

## Furthest sum algorithm

The furthest sum algorithm is one of two choices for an algorithm to generate archetypes. It is the one used by default. This can be found in `furthest_sum.py` in `aleixalcacer`'s repository.

As far as I can tell, this can be left unchanged. It should work regardless of the data type being reduced.

In [3]:
def furthest_sum(K, noc, random_state):
    """Furthest sum algorithm, to efficiently generate initial seed/archetypes.

    Soruce: https://github.com/ulfaslak/py_pcha/blob/master/py_pcha/furthest_sum.py

    Note: Commonly data is formatted to have shape (examples, dimensions).
    This function takes input and returns output of the transposed shape,
    (dimensions, examples).

    Parameters
    ----------
    K : np.ndarray
        Either a data matrix or a kernel matrix of two dimensions.

    noc : int
        Number of archetypes to extract.

    random_state: np.random_state
        Random generator.
    Output
    ------
    i : List[int]
        The extracted candidate archetypes
    """

    def max_ind_val(v):
        return max(zip(range(len(v)), v), key=lambda x: x[1])

    i_shape, j_shape = K.shape
    i = [int(np.floor(j_shape * random_state.rand()))]
    index = np.array(range(j_shape))
    index[i] = -1
    ind_t = i
    sum_dist = np.zeros((1, j_shape), np.complex128)

    if j_shape > noc * i_shape:
        Kt = K
        Kt2 = np.sum(Kt**2, axis=0)
        for k in range(1, noc + 11):
            if k > noc - 1:
                Kq = np.dot(Kt[:, i[0]], Kt)
                sum_dist -= np.lib.scimath.sqrt(Kt2 - 2 * Kq + Kt2[i[0]])
                index[i[0]] = i[0]
                i = i[1:]
            t = np.where(index != -1)[0]
            Kq = np.dot(Kt[:, ind_t].T, Kt)
            sum_dist += np.lib.scimath.sqrt(Kt2 - 2 * Kq + Kt2[ind_t])
            ind, val = max_ind_val(sum_dist[:, t][0].real)
            ind_t = t[ind]
            i.append(ind_t)
            index[ind_t] = -1
    else:
        if i_shape != j_shape or np.sum(K - K.T) != 0:  # Generate kernel if K not one
            Kt = K
            K = np.dot(Kt.T, Kt)
            K = np.lib.scimath.sqrt(
                np.tile(np.diag(K), (j_shape, 1))
                - 2 * K
                + np.tile(np.mat(np.diag(K)).T, (1, j_shape))
            )

        Kt2 = np.diag(K)  # Horizontal
        for k in range(1, noc + 11):
            if k > noc - 1:
                sum_dist -= np.lib.scimath.sqrt(Kt2 - 2 * K[i[0], :] + Kt2[i[0]])
                index[i[0]] = i[0]
                i = i[1:]
            t = np.where(index != -1)[0]
            sum_dist += np.lib.scimath.sqrt(Kt2 - 2 * K[ind_t, :] + Kt2[ind_t])
            ind, val = max_ind_val(sum_dist[:, t][0].real)
            ind_t = t[ind]
            i.append(ind_t)
            index[ind_t] = -1
    return i


## Alternating Optimisation

Because the archetype analysis problem involves optimising over both $\alpha$ and $\beta$, alternating optimisation is used to solve the problem. I use the same optimisation procedure as in `aleixalcacer`'s code, assuming that there is no real difference in data types for optimising $\alpha$ and $\beta$.

### Optimising for $\alpha$

In [4]:
def _optimize_alphas(B, A):
    B = np.pad(B, ((0, 0), (0, 1)), "constant", constant_values=200)
    A = np.pad(A, ((0, 0), (0, 1)), "constant", constant_values=200)
    alphas = np.empty((B.shape[0], A.shape[0]))
    for j in range(alphas.T.shape[1]):
        alphas.T[:, j], _ = nnls(A.T, B.T[:, j])
    alphas /= alphas.sum(1)[:, None]
    alphas[np.isnan(alphas)] = 1 / alphas.shape[1]
    return alphas

### Optimising for $\beta$

In [5]:
def _optimize_betas(B, A):
    return _optimize_alphas(B, A)

### The loss update step

This is where the difference occurs. The code is `aleixalcacer`'s repository needs to be updated to account for multiple types of data.

To adapt this code, I propose adding two new parameters to the `_aa_simple()` function defined in `archetypes.py`. The first new parameter is `mode_cuts`. This parameter is a list of ranges, and must be of length three.

The first range in `mode_cuts`, $\mathcal{N}_1$, contains the indices of every numeric feature in the data. These features will have $L_{\text{real}}$ applied to them.

The second range in `mode_cuts`, $\mathcal{N}_2$, contains the indices of every Boolean feature in the data. These features will have $L_{\text{bool}}$ applied to them.

The third range in `mode_cuts`, $\mathcal{N}_3$, contains the indices of every ordinal feature in the data. These features will have $L_{\text{ord}}$ applied to them.

All of the ranges in `mode_cuts` must be subsets of $[1,m]$, where $m$ is the number of features in the data.

The second new parameter is `ord_card`. This parameter is necessary because each ordinal feature in the dataset can have different cardinalities, that is, a different number of choices. For calculating the modified hinge loss, it's necessary to have access to each cardinality.

The length of `ord_card` has to be same as the length of the third range in `mode_cuts`.

In [6]:
def _aa_mixed(X, i_alphas, i_betas, max_iter, tol, mode_cuts, ord_card, verbose=False):
    for mode_cut in mode_cuts:
        if len(mode_cut) > 0:
            assert np.min(mode_cut) >= 0 and np.max(mode_cut) < X.shape[1]
    alphas = i_alphas
    betas = i_betas
    n = X.shape[0]

    Z = betas @ X

    loss_0 = inf
    for n_iter in range(max_iter):
        if verbose and n_iter % 100 == 0:
            print(f"    Iteration: {n_iter + 1:{len(str(max_iter))}}, Loss: {loss_0:.2f}")

        alphas = _optimize_alphas(X, Z)
        Z = np.linalg.pinv(alphas) @ X
        betas = _optimize_betas(Z, X)
        Z = betas @ X
        loss_real = 0
        if len(mode_cuts[0]) > 0:
            X_real = X[:, mode_cuts[0]] # the indices of the input data which contain numeric values
            Z_real = Z[:, mode_cuts[0]] # the corresponding indices for X_real among the archetypes
            loss_real = np.linalg.norm(X_real - alphas @ Z_real) # Frobenius norm
        loss_bool = 0
        if len(mode_cuts[1]) > 0:
            X_bool = X[:, mode_cuts[1]] # the indices of the input data which contain Boolean values
            Z_bool = Z[:, mode_cuts[1]] # the corresponding indices for X_bool among the archetypes
            zeros = np.zeros(X_bool.shape[1])
            ones = np.ones(X_bool.shape[1])
            alpha_Z = alphas @ Z_bool
            hinge = X_bool.T @ alpha_Z
            loss_bool = np.sum(np.maximum(zeros, ones - hinge)) # Hinge loss
        loss_ord = 0
        for i, x in enumerate(mode_cuts[2]): # Modified hinge loss for ordinal data
            # TODO: find a more efficient version of this calculation using np.
            # This is way too computationally expensive!
            X_ord = np.tile(X[:, x], (ord_card[i], 1)).T
            Z_ord = np.tile(Z[:, x], (ord_card[i], 1)).T # Get the archetypal feature corresponding to each ordinal input data
            d = np.arange(1, ord_card[i] + 1) # The number of categories in the ordinal data
            # I think calculating S is a simpler way to perform the summation in GLRM - use the sign as a
            # multiplier to cover both cases within L_{ord}. Then subtract 1 to handle the case where the
            # sign is 0
            S = np.sign(np.broadcast_to(d, (X_ord.shape[0], ord_card[i])) - X_ord)
            zeros = np.zeros(ord_card[i]) # Hinge loss must be non-negative
            ones = np.ones(ord_card[i])
            Z_alpha = alphas @ Z_ord
            D = np.broadcast_to(d, Z_alpha.shape)
            diff = Z_alpha - D
            signed_diff = S * diff
            # Calculate the modified hinge loss for one feature!
            loss_ord += np.sum(np.maximum(zeros, ones + signed_diff)) - 1
        loss = loss_real + loss_bool + loss_ord
        if np.abs(loss_0 - loss) < tol:
            break
        loss_0 = loss

    return alphas, betas, loss_0, Z, n_iter

### Time for the archetypal analysis model!

This will essentially be the same as in `aleixalcacer`'s code.

In [7]:
class AA(BaseEstimator, TransformerMixin):
    """
    Archetype Analysis estimator.

    Parameters
    ----------
    n_archetypes : int, default=4
        The number of archetypes to compute.
    n_init : int, default=5
         Number of time the archetype analysis algorithm will be run with
         different coefficient initialization. The final results will be the
         best output of *n_init* consecutive runs in terms of RSS.
    max_iter : int, default=300
        Maximum number of iterations of the archetype analysis algorithm
        for a single run.
    tol : float, default=1e-4
        Relative tolerance of two consecutive iterations to declare convergence.
    verbose : bool, default=False
        Verbosity mode.
    random_state : int, RandomState instance or None, default=None
        Determines random number generation of coefficients. Use an int to make
        the randomness deterministic.

    References
    ----------
    .. [1] Adele Cutler, & Leo Breiman (1994). Archetypal analysis.
       Technometrics, 36, 338-347.


    """

    def __init__(
        self,
        mode_cuts,
        ord_card,
        n_archetypes=4,
        n_init=1,
        max_iter=300,
        tol=1e-4,
        algorithm_init="auto",
        verbose=False,
        random_state=None,
    ):
        self.mode_cuts = mode_cuts
        self.ord_card = ord_card
        self.n_archetypes = n_archetypes
        self.max_iter = max_iter
        self.tol = tol
        self.n_init = n_init
        self.verbose = verbose
        self.random_state = random_state
        self.algorithm_init = algorithm_init

    def _check_data(self, X):
        if X.shape[0] < self.n_archetypes:
            raise ValueError(
                f"n_samples={X.shape[0]} should be >= n_archetypes={self.n_archetypes}."
            )

    def _check_parameters(self):
        if not isinstance(self.n_archetypes, int):
            raise TypeError
        if self.n_archetypes <= 0:
            raise ValueError(f"n_archetypes should be > 0, got {self.n_archetypes} instead.")

        if not isinstance(self.max_iter, int):
            raise TypeError
        if self.max_iter <= 0:
            raise ValueError(f"max_iter should be > 0, got {self.max_iter} instead.")

        if not isinstance(self.n_init, int):
            raise TypeError
        if self.n_init <= 0:
            raise ValueError(f"n_int should be > 0, got {self.n_init} instead.")

        if not isinstance(self.algorithm_init, str):
            raise TypeError
        algorithm_init_names = ["auto", "random", "furthest_sum"]
        if self.algorithm_init not in algorithm_init_names:
            raise ValueError(
                f"algorithm_init must be one of {algorithm_init_names}, "
                f"got {self.algorithm_init} instead."
            )
        self._algorithm_init = self.algorithm_init

        if self._algorithm_init == "auto":
            self._algorithm_init = "furthest_sum"

        if not isinstance(self.verbose, bool):
            raise TypeError
            
        if mode_cuts.shape[0] != 3:
            raise ValueError(f"mode_cuts should be of length 3!")
        assert self.ord_card.shape[0] == self.mode_cuts[2].shape[0]

    def _init_coefs(self, X, random_state):
        ind = random_state.choice(self.n_archetypes, X.shape[0])
        alphas = np.zeros((X.shape[0], self.n_archetypes), dtype=np.float64)
        for i, j in enumerate(ind):
            alphas[i, j] = 1

        betas = np.zeros((self.n_archetypes, X.shape[0]), dtype=np.float64)
        if self._algorithm_init == "random":
            ind = random_state.choice(X.shape[0], self.n_archetypes)
        else:
            ind = furthest_sum(X.T, self.n_archetypes, random_state)
        for i, j in enumerate(ind):
            betas[i, j] = 1

        return alphas, betas

    def fit(self, X, y=None, **fit_params):
        """
        Compute Archetype Analysis.

        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Training instances to compute the archetypes.
            It must be noted that the data will be converted to C ordering,
            which will cause a memory copy if the given data is not C-contiguous.
            If a sparse matrix is passed, a copy will be made if it's not in
            CSR format.
        y : Ignored
            Not used, present here for API consistency by convention.

        Returns
        -------
        self : object
            Fitted estimator.
        """
        X = self._validate_data(X, dtype=[np.float64, np.float32])
        self._check_parameters()
        self._check_data(X)
        random_state = check_random_state(self.random_state)

        self.rss_ = inf
        for i in range(self.n_init):
            if self.verbose:
                print(f"Initialization {i + 1:{len(str(self.n_init))}}/{self.n_init}")

            i_alphas, i_betas = self._init_coefs(X, random_state)

            alphas, betas, rss, Z, n_iter = _aa_mixed(
                X, i_alphas, i_betas, self.max_iter, self.tol, self.mode_cuts, self.ord_card, self.verbose
            )

            if rss < self.rss_:
                self.alphas_ = alphas
                self.betas_ = betas
                self.archetypes_ = Z
                self.n_iter_ = n_iter
                self.rss_ = rss

        return self

    def transform(self, X):
        """
        Transform X to an archetype-distance space.

        In the new space, each dimension is the distance to the archetypes.
        Note that even if X is sparse, the array returned by `transform` will
        typically be dense.

        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            New data to transform.

        Returns
        -------
        X_new : ndarray of shape (n_samples, n_archetypes)
            X transformed in the new space.
        """
        check_is_fitted(self)
        X = self._validate_data(X, dtype=[np.float64, np.float32])
        self._check_parameters()
        Z = self.archetypes_
        alphas = _optimize_alphas(X, Z)
        return alphas

    def fit_transform(self, X, y=None, **fit_params):
        """
        Compute the archetypes and transform X to archetype-distance space.

        Equivalent to fit(X).transform(X), but more efficiently implemented.

        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            New data to transform.
        y : Ignored
            Not used, present here for API consistency by convention.

        Returns
        -------
        X_new : ndarray of shape (n_samples, n_archetypes)
            X transformed in the new space.
        """
        return self.fit(X, y, **fit_params).transform(X)


## Experiments time!

### Small experiment with just numerical data

Just to check I haven't destroyed anything fundamentally in the process of changing the archetypal analysis model!

In [8]:
X = np.random.normal(0, 1, (100, 2))

mode_cuts = np.array([np.array(range(2)), np.array([]), np.array([])], dtype=object)
ord_card = np.array([])

aa = AA(mode_cuts, ord_card, n_archetypes=4)

X_trans = aa.fit_transform(X)

### Did this work?

Let's see if the archetypes match our expectations...

In [9]:
aa.archetypes_

array([[ 1.63515943,  2.15199346],
       [-1.16584677, -1.92676026],
       [ 1.96768238, -1.78853253],
       [-2.55065393,  0.40752317]])

## More complex data

Let's try the Adult data set now - this has numerical, boolean, categorical and ordinal data!

For more detail on the Adult data set, feel free to have a look at [this website](https://search.r-project.org/CRAN/refmans/arules/html/Adult.html).

In [19]:
names = np.array(['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'yearly-income'])
adult_data = pd.read_csv(path_to_data, names=names, index_col=False)
adult_data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,yearly-income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


### Clean up the data

First of all, I want to rearrange the data such that all the numerical data is first, the Boolean/categorical data is second and the ordinal data is third.

In [20]:
# All the numeric data in the Adult data set
real_data = ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
# All the boolean data in the Adult data set
bool_data = ['sex', 'yearly-income']
# All the categorical data in the Adult data set
cat_data = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']
# All the ordinal data in the Adult data set
ord_data = ['education-num']
# The order of the education field of the data
education_order = ['Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th', '11th', '12th', 'HS-grad', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Some-college', 'Bachelors', 'Masters', 'Doctorate']

The categorical data is represented via one-hot encodings in the data. That is, each value for each categorical variable becomes its own column in the modified data.

To convert back to the categorical variables (or at least group them for analysis after obtaining the archetypes), I need to store the relationship between categorical variables and their values.

In [21]:
# A correspondence from categorical variables to their values will be useful after we get our archetypes
cat_vars = dict()
for cat in cat_data:
    cat_vars[cat] = np.unique(adult_data[cat])

I need to modify the boolean data with `True` or `False`.

In [22]:
# Replace boolean data with 'True'/'False'
for b in bool_data:
    vals = np.unique(adult_data[b])
    true_val = vals[0]
    false_val = vals[1]
    print('Correlation for Boolean category', b)
    print(f'{true_val} -> True, {false_val} -> False')
    adult_data[b] = adult_data[b] == true_val

Correlation for Boolean category sex
 Female -> True,  Male -> False
Correlation for Boolean category yearly-income
 <=50K -> True,  >50K -> False


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,yearly-income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,False,2174,0,40,United-States,True
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,False,0,0,13,United-States,True
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,False,0,0,40,United-States,True
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,False,0,0,40,United-States,True
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,True,0,0,40,Cuba,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,True,0,0,38,United-States,True
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,False,0,0,40,United-States,False
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,True,0,0,40,United-States,True
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,False,0,0,20,United-States,True


I need to also change the categorical data into one-hot encoding data as mentioned above.

In [23]:
# Convert all the categorical data into a one-hot encoding
for cat in cat_data:
    unique_vals = np.unique(adult_data[cat])
    for unique_val in unique_vals:
        if unique_val != ' ?':
            one_hot_data = adult_data[cat] == unique_val
            adult_data[unique_val] = one_hot_data
            bool_data.append(unique_val)
    adult_data.drop(cat, axis=1, inplace=True)

,age,fnlwgt,education,education-num,sex,capital-gain,capital-loss,hours-per-week,yearly-income,Federal-gov,...,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
0,39,77516,Bachelors,13,False,2174,0,40,True,False,...,False,False,False,False,False,False,False,True,False,False
1,50,83311,Bachelors,13,False,0,0,13,True,False,...,False,False,False,False,False,False,False,True,False,False
2,38,215646,HS-grad,9,False,0,0,40,True,False,...,False,False,False,False,False,False,False,True,False,False
3,53,234721,11th,7,False,0,0,40,True,False,...,False,False,False,False,False,False,False,True,False,False
4,28,338409,Bachelors,13,True,0,0,40,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,Assoc-acdm,12,True,0,0,38,True,False,...,False,False,False,False,False,False,False,True,False,False
32557,40,154374,HS-grad,9,False,0,0,40,False,False,...,False,False,False,False,False,False,False,True,False,False
32558,58,151910,HS-grad,9,True,0,0,40,True,False,...,False,False,False,False,False,False,False,True,False,False
32559,22,201490,HS-grad,9,False,0,0,20,True,False,...,False,False,False,False,False,False,False,True,False,False


I then need to re-arrange the data so that all the numerical data precedes the boolean/one-hot data, which precedes the ordinal data.

In [24]:
# Rearrange the data so that the columns are of the form real data -> boolean data -> ordinal data
cols = real_data + bool_data + ord_data
adult_data = adult_data[cols]

,age,fnlwgt,capital-gain,capital-loss,hours-per-week,sex,yearly-income,Federal-gov,Local-gov,Never-worked,...,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia,education-num
0,39,77516,2174,0,40,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,13
1,50,83311,0,0,13,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,13
2,38,215646,0,0,40,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,9
3,53,234721,0,0,40,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,7
4,28,338409,0,0,40,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,0,0,38,True,True,False,False,False,...,False,False,False,False,False,False,True,False,False,12
32557,40,154374,0,0,40,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,9
32558,58,151910,0,0,40,True,True,False,False,False,...,False,False,False,False,False,False,True,False,False,9
32559,22,201490,0,0,20,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,9


Finally, I'm going to drop whichever category corresponds to the proxy I'm looking for. By default this is `sex` but feel free to change it to any category you like.

In [25]:
# It seems ready to go!
# First need to drop the category I want to find as a proxy
cat_to_drop = 'sex' # change this to explore other protected categories!
adult_data = adult_data.drop(cat_to_drop, axis=1)
bool_data.remove(cat_to_drop)
adult_data

,age,fnlwgt,capital-gain,capital-loss,hours-per-week,yearly-income,Federal-gov,Local-gov,Never-worked,Private,...,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia,education-num
0,39,77516,2174,0,40,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,13
1,50,83311,0,0,13,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,13
2,38,215646,0,0,40,True,False,False,False,True,...,False,False,False,False,False,False,True,False,False,9
3,53,234721,0,0,40,True,False,False,False,True,...,False,False,False,False,False,False,True,False,False,7
4,28,338409,0,0,40,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,0,0,38,True,False,False,False,True,...,False,False,False,False,False,False,True,False,False,12
32557,40,154374,0,0,40,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,9
32558,58,151910,0,0,40,True,False,False,False,True,...,False,False,False,False,False,False,True,False,False,9
32559,22,201490,0,0,20,True,False,False,False,True,...,False,False,False,False,False,False,True,False,False,9


### Setting up the model for complex data

Now that the data is properly cleaned, I set up the Archetype Analysis model.

In [27]:
real_cols = len(real_data)
bool_cols = len(bool_data)
ord_cols = len(ord_data)
mode_cuts = np.array([np.arange(real_cols), np.arange(real_cols, real_cols + bool_cols), np.arange(real_cols + bool_cols, real_cols + bool_cols + ord_cols)], dtype=object)
aa = AA(mode_cuts, np.full(1, 13), n_archetypes=2)

Now it's time to fit the Archetype Analysis model to the Adult dataset.

In [28]:
adult_data.columns = adult_data.columns.astype(str) # solves an error that occurred before where the type of each column was different
adult_data_trans = aa.fit_transform(adult_data)
# This is the transformed data - it's represented in terms of the archetypes.
# So its dimensionality will be (n, |A|), where n is the number of data points and |A| is the number of archetypes
adult_data_trans

These are the archetypes. Let's set them up as a `DataFrame` and do some further analysis.

In [30]:
archetypes = pd.DataFrame(aa.archetypes_, columns=adult_data.columns)
archetypes

,age,fnlwgt,capital-gain,capital-loss,hours-per-week,yearly-income,Federal-gov,Local-gov,Never-worked,Private,...,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia,education-num
0,38.338277,191293.406405,0.0,90.253581,40.245587,0.779693,0.029346,0.064594,0.000222,0.700816,...,0.003588,0.000373,0.00242,0.001569,0.000571,0.000603,0.895216,0.002071,0.000501,10.015946
1,53.658312,37999.508279,99999.0,0.000000,50.915817,0.000000,0.000000,0.000000,0.000000,0.395763,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.995221,0.000000,0.000000,14.313233


A few interesting observations with regards to the numerical, boolean and ordinal data:
1. Both archetypes in this example are fairly highly educated. Archetype #1 has an education level of 'Some-College', which I assume corresponds to having graduated from community college. Archetype #2 has a Master's degree.
2. Archetype #1 is approximately 38 years old, whereas archetype #2 is between 53 and 54 years old.
3. Archetype #1 has no capital gain, and archetype #2 has immense capital gain. I suspect this variable is subject to outliers.
4. There is similar story with regards to capital loss as there is for capital gain.
5. Archetype #1 is fairly likely to earn more than $50K, whereas archetype #2 has absolutely no chance of earning that much money.
6. Both archetypes work at least full-time hours.

### Analysis of categorical variables

Let's now look at the categorical variables in more detail. Recall the correspondences between each categorical variable and its values:

In [31]:
cat_vars

{'workclass': array([' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private',
        ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
       dtype=object),
 'marital-status': array([' Divorced', ' Married-AF-spouse', ' Married-civ-spouse',
        ' Married-spouse-absent', ' Never-married', ' Separated',
        ' Widowed'], dtype=object),
 'occupation': array([' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair',
        ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners',
        ' Machine-op-inspct', ' Other-service', ' Priv-house-serv',
        ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support',
        ' Transport-moving'], dtype=object),
 'relationship': array([' Husband', ' Not-in-family', ' Other-relative', ' Own-child',
        ' Unmarried', ' Wife'], dtype=object),
 'race': array([' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other',
        ' White'], dtype=object),
 'native-country': array([' ?', ' Cambodia'

#### Work class

Let's first look at what kind of capacity both archetypes work in:

In [32]:
archetypes[[work for work in cat_vars['workclass'] if work != ' ?']]

,Federal-gov,Local-gov,Never-worked,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay
0,0.029346,0.064594,0.000222,0.700816,0.031068,0.076180,0.040148,0.000439
1,0.000000,0.000000,0.000000,0.395763,0.382043,0.222194,0.000000,0.000000


Taking a simple maximum as a heuristic, archetype #1 is most likely to work in the private sector. Archetype #2, on the other hand, is almost equally likely to work in the private sector or be self-employed. Recall that archetype #2 is completely unlikely to earn more than $50K and has a Master's degree.

#### Martial status

Let's look at the marital status of each archetype:

In [33]:
archetypes[[marital_status for marital_status in cat_vars['marital-status']]]

,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed
0,0.137723,0.000691,0.450663,0.012954,0.335074,0.032012,0.030883
1,0.000000,0.000000,0.782586,0.000000,0.217414,0.000000,0.000000


Archetype #2 is fairly likely to be married, but may also be not married. Archetype #1 is more likely to not be married, but there is also a 45% chance they are married.

#### Occupation

Let's see the distribution of occupations for each archetype:

In [34]:
archetypes[[occupation for occupation in cat_vars['occupation'] if occupation != ' ?']]

,Adm-clerical,Armed-Forces,Craft-repair,Exec-managerial,Farming-fishing,Handlers-cleaners,Machine-op-inspct,Other-service,Priv-house-serv,Prof-specialty,Protective-serv,Sales,Tech-support,Transport-moving
0,0.117683,0.000286,0.127263,0.120345,0.030515,0.043137,0.062837,0.103866,0.004676,0.121663,0.020212,0.111648,0.028814,0.049648
1,0.000000,0.000000,0.000000,0.382043,0.000000,0.000000,0.000000,0.000000,0.000000,0.617957,0.000000,0.000000,0.000000,0.000000


The distribution for archetype #1 is spread quite thin. The front-runners, in order from (marginally) more to less likely occupations, are:
- `Craft-repair`;
- `Prof-specialty`;
- `Exec-managerical`;
- `Adm-clerical`;
- `Sales`; and
- `Other-service`.
Recall that this archetype most likely works in the private sector.

The other archetype was very likely to work in `Prof-specialty` or `Exec-managerial`. This aligns with the possibly that archetype #2 is either privately or self-employed.

#### Relationship

Let's look at the relationship status of each archetype:

In [35]:
archetypes[[relationship for relationship in cat_vars['relationship']]]

,Husband,Not-in-family,Other-relative,Own-child,Unmarried,Wife
0,0.396738,0.257294,0.030821,0.160055,0.107846,0.047246
1,0.782586,0.217414,0.000000,0.000000,0.000000,0.000000


Both archetypes seem most likely to be husbands - this variable is a fairly telling proxy for `sex`, the protected characteristic that I removed from the data.

#### Race

Let's look at the ethnicity/race of each archetype:

In [36]:
archetypes[[race for race in cat_vars['race']]]

,Amer-Indian-Eskimo,Asian-Pac-Islander,Black,Other,White
0,0.009545,0.03165,0.097612,0.008417,0.852776
1,0.000000,0.00000,0.000000,0.004779,0.995221


Both archetypes seem quite likely to be white.

#### Country of origin

Let's look at the country of origin for each archetype:

In [37]:
archetypes[[native_country for native_country in cat_vars['native-country'] if native_country != ' ?']]

,Cambodia,Canada,China,Columbia,Cuba,Dominican-Republic,Ecuador,El-Salvador,England,France,...,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
0,0.000573,0.003652,0.002343,0.001864,0.002993,0.002165,0.000873,0.003269,0.002778,0.000892,...,0.001165,0.003588,0.000373,0.00242,0.001569,0.000571,0.000603,0.895216,0.002071,0.000501
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.995221,0.000000,0.000000


Both archetypes are very likely to have been raised in the United States.

## Next steps

1. Use more archetypes in my analysis to see if the number of archetypes determines the diversity of the archetypes - these archetypes aren't particularly diverse!
2. Do some more data modification. [This description of the data set](https://search.r-project.org/CRAN/refmans/arules/html/Adult.html) converted a lot of the numeric variables into ordinal data. This might be useful for creating archetypes.